**Tanish Chhabra<br>
101916038<br>
CS10**

In [1]:
import cupy as cp
import cudf
import cuml

In [2]:
train = cudf.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = cudf.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
Y = train['SalePrice']
train.drop('SalePrice',axis=1,inplace=True)

In [4]:
n_train,n_test = train.shape[0],test.shape[0]
train.shape,test.shape

((1460, 80), (1459, 80))

In [5]:
df = cudf.concat([train,test],axis=0)
df.shape

(2919, 80)

In [6]:
# Drop columns having more than 25% missing values
to_drop = []
categorical_features = []
for col in df.keys():
    if df[col].dtype == 'object':
        categorical_features.append(col)
        
    if df[col].isnull().sum()>=len(train)//4:
        to_drop.append(col)

df = df.drop(to_drop,axis=1)

In [7]:
for col in df.keys():
    if df[col].isnull().sum()!=0:
        if col in categorical_features:
            df[col].fillna(df[col].mode()[0],inplace=True)
        else:
            df[col].fillna(df[col].mean(),inplace=True)

In [8]:
df.isnull().any().sum()

0

In [9]:
df = cudf.get_dummies(df)

In [10]:
train = df.iloc[:n_train,:]
test = df.iloc[n_test+1:,:]

In [11]:
train.shape,test.shape

((1460, 270), (1459, 270))

In [12]:
X = train

In [13]:
X_train,X_validate,Y_train,Y_validate = cuml.train_test_split(X,Y,test_size=0.3,random_state=42)

In [14]:
algorithms = ['svd', 'eig', 'svd-qr', 'svd-jacobi']
for algo in algorithms:
    model = cuml.LinearRegression(algorithm=algo)
    trained = model.fit(X_train,Y_train)
    Y_predict = trained.predict(X_validate)
    
    mse = cuml.metrics.regression.mean_squared_error(Y_validate.astype('int64'),Y_predict.astype('int64'))
    mae = cuml.metrics.regression.mean_absolute_error(Y_validate.astype('int64'),Y_predict.astype('int64'))
    r2 = cuml.metrics.regression.r2_score(Y_validate.astype('float64'),Y_predict.astype('float64'))
    
    print("Algorithm: "+algo)
    print("R2 score:",str(r2),"      MSE:",str(mse),"       MAE:",str(mae))
    print()

Algorithm: svd
R2 score: 0.8442217624049219       MSE: 920522253.7762557        MAE: 19054.56621004566

Algorithm: eig
R2 score: -1.8555127121245722       MSE: 16873734209.646118        MAE: 38892.824200913245

Algorithm: svd-qr
R2 score: 0.8494382633663469       MSE: 889697252.630137        MAE: 19203.109589041094

Algorithm: svd-jacobi
R2 score: 0.8442217624049219       MSE: 920522253.7762557        MAE: 19054.56621004566



In [15]:
model = cuml.LinearRegression(algorithm='svd-qr')
trained = model.fit(X,Y)
Y_predict = trained.predict(test)

In [16]:
Y_predict

0       113102.264500
1       163144.344321
2       190188.427516
3       196269.428041
4       203270.797196
            ...      
1454     81770.541706
1455     83043.579672
1456    177436.923970
1457    106972.740238
1458    222976.846527
Length: 1459, dtype: float64

In [17]:
output = cudf.DataFrame({'Id': test.Id, 'SalePrice': Y_predict})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [18]:
output

,Id,SalePrice
0,1461,113102.264500
1,1462,163144.344321
2,1463,190188.427516
3,1464,196269.428041
4,1465,203270.797196
...,...,...
1454,2915,81770.541706
1455,2916,83043.579672
1456,2917,177436.923970
1457,2918,106972.740238
